In [ ]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import requests

In [ ]:
#format list for the dataframe
key = '6924a24c5e8fd68ab4c75b8911dde245'
state_ids = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL',
             'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
             'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE',
             'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK',
             'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT',
             'VA', 'WA', 'WI', 'WV', 'WY']

In [ ]:
#create dataframe and read in the two json files holding personal expenditures and personal income per capita
for x in state_ids:
  RINCPC = f"https://api.stlouisfed.org/fred/series/observations?series_id={x}RPIPC&api_key={key}&file_type=json"
  response = requests.get(RINCPC)
  series = pd.DataFrame(response.json()["observations"])
  series["value"] = series.rename(columns={'value': x}, inplace=True)
  series = series[['date', x]]
  if x == state_ids[0]:
    df = series
  else:
    df = pd.merge(df, series, on='date', how='inner')

In [ ]:
# Database connection configuration
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'localhost'
USER = 'glrod'
PASSWORD = 'team3'
PORT = 5432
DATABASE = 'healthcare_db'  

In [ ]:
# Create the connection string
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"

In [ ]:
# Create the database engine
engine = create_engine(connection_string)


In [ ]:
# Load CSV files into dataframes
hospitals_df = pd.read_csv("C:\\Users\\gezeu\\Project_3\\Hospital.csv")
physicians_df = pd.read_csv("C:\\Users\\gezeu\\Project_3\\Physician and Clinics.csv")
realincome_df = pd.read_csv("C:\\Users\\gezeu\\Project_3\\realincome.csv")

In [ ]:
# Load data into PostgreSQL database
hospitals_df.to_sql('Hospitals', con=engine, if_exists='replace', index=False)
physicians_df.to_sql('Physicians_and_Clinics', con=engine, if_exists='replace', index=False)
realincome_df.to_sql('RealIncome', con=engine, if_exists='replace', index=False)

In [ ]:
# Query data from PostgreSQL
hospitals_query = "SELECT * FROM \"Hospitals\""
physicians_query = "SELECT * FROM \"Physicians_and_Clinics\""
realincome_query = "SELECT * FROM \"RealIncome\""


hospitals_df = pd.read_sql(hospitals_query, con=engine)
physicians_df = pd.read_sql(physicians_query, con=engine)
realincome_df = pd.read_sql(realincome_query, con=engine)


In [ ]:
# Pivot the real income data to match the format of other tables (pivoting)
# Adjust melt operation based on available columns
realincome_df = realincome_df.melt(id_vars=['Year'], var_name='state', value_name='income')
realincome_df = realincome_df.pivot(index='Year', columns='state', values='income').reset_index()


In [ ]:
# Merge dataframes
final_df = pd.merge(hospitals_df, physicians_df, on='Region/state of residence', suffixes=('_hospital', '_physician'))
final_df = pd.merge(final_df, realincome_df, left_on='Region/state of residence', right_on='Year')

In [ ]:
# Filter only states (assuming 'Region/state of residence' column contains state names)
states = set(['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WI', 'WV', 'WY'])
final_df = final_df[final_df['Region/state of residence'].isin(states)]


In [ ]:
# Melt the dataframe for easier plotting
melted_df = final_df.melt(id_vars=['Region/state of residence'],
                          value_vars=['2008_hospital', '2008_physician', '2008_income',
                                      '2009_hospital', '2009_physician', '2009_income',
                                      '2010_hospital', '2010_physician', '2010_income',
                                      '2011_hospital', '2011_physician', '2011_income',
                                      '2012_hospital', '2012_physician', '2012_income',
                                      '2013_hospital', '2013_physician', '2013_income',
                                      '2014_hospital', '2014_physician', '2014_income',
                                      '2015_hospital', '2015_physician', '2015_income',
                                      '2016_hospital', '2016_physician', '2016_income',
                                      '2017_hospital', '2017_physician', '2017_income',
                                      '2018_hospital', '2018_physician', '2018_income',
                                      '2019_hospital', '2019_physician', '2019_income',
                                      '2020_hospital', '2020_physician', '2020_income'],
                          var_name='category_year', value_name='value')

melted_df = final_df.melt(id_vars=['Region/state of residence'],
                          value_vars=['2008_hospital', '2008_physician', '2008_income',
                                      '2009_hospital', '2009_physician', '2009_income',
                                      # Adjust the list based on the actual column names
                                      '2010_hospital', '2010_physician', '2010_income'],
                          var_name='category_year', value_name='value')




In [ ]:
# Split 'category_year' into 'Category' and 'Year'
melted_df[['Year', 'Category']] = melted_df['category_year'].str.extract(r'(\d{4})_(hospital|physician|income)')
melted_df.drop(columns=['category_year'], inplace=True)


In [ ]:
# Plotting grouped bar chart
def plot_grouped_bar_chart(state):
    state_df = melted_df[melted_df['Region/state of residence'] == state]
    state_df = state_df.pivot(index='Year', columns='Category', values='value').reset_index()

    categories = ['hospital', 'physician', 'income']
    bar_width = 0.2
    years = state_df['Year'].unique()
    x = np.arange(len(years))

    fig, ax = plt.subplots(figsize=(14, 8))

    for i, category in enumerate(categories):
        ax.bar(x + i * bar_width, state_df[category], bar_width, label=category.capitalize())

    ax.set_xlabel('Year')
    ax.set_ylabel('Value')
    ax.set_title(f'Health Expenditure and Real Income in {state}')
    ax.set_xticks(x + bar_width)
    ax.set_xticklabels(years)
    ax.legend()

    plt.show()


In [ ]:
plot_grouped_bar_chart('CA')  # Replace 'CA' with any state abbreviation you want to plot
